In [5]:
import tqdm
import csv
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch
import torch.optim as optim
from torch.autograd import Variable
import tqdm
import numpy as np
from FCN import FullyConnectedNet
import torch.nn as nn

In [9]:
# dataset define
class Dataset(Dataset):
    def __init__(self, method):
        self.root = '/Users/daehyeon/Desktop/수업/컴퓨터비전/final/data/' #Change this path
        self.x_data = []
        self.y_data = []
        if method == 'train':
            self.root = self.root + 'train_data.csv'
            with open(self.root, newline='') as csvfile:
                data_reader = csv.reader(csvfile, delimiter=',')
                data = list(data_reader)
                for i in range(len(data)):
                    x_float_data = list(map(float, data[i][:-1])) # String list to float list
                    self.x_data.append(x_float_data)
                    
                    self.y_data.append(int(data[i][-1]))
                    

        elif method == 'val':
            self.root = self.root + 'val_data.csv'
            with open(self.root, newline='') as csvfile:
                data_reader = csv.reader(csvfile, delimiter=',')
                data = list(data_reader)
                for i in range(len(data)):
                    x_float_data = list(map(float, data[i][:-1])) # String list to float list
                    self.x_data.append(x_float_data)
                    
                    self.y_data.append(int(data[i][-1]))
                    
    def _one_hot_encoding(self, index):
        result = [0, 0, 0]
        result[index] = 1
        return result
    
    def __len__(self):
        return len(self.x_data)

    def __getitem__(self, idx):
        return torch.FloatTensor(self.x_data[idx]), torch.from_numpy(np.array(self.y_data[idx]))


In [23]:
# hyperparameters
epochs = 100
batch_size = 50
learning_rate = 0.00005

In [10]:
# train
net = FullyConnectedNet()

dataset = Dataset(method='train')
dataloader = DataLoader(dataset, batch_size=batch_size)
weight_path = "/Users/daehyeon/Desktop/수업/컴퓨터비전/final/weights/"

loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)

# learning rate scheduler helps decaying the learning rate while training a model.
decayRate = 0.9999
lr_scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer=optimizer, gamma=decayRate)

for epoch in tqdm.tqdm(range(epochs + 1)):
    
    for batch_idx, samples in enumerate(dataloader):
        x_train, y_train = samples
        
        net_out = net(x_train)
        loss = loss_func(net_out, y_train)

        loss.backward()
        optimizer.step()
        lr_scheduler.step()
    
    
    # val for every epoch
    val_dataset = Dataset(method='val')
    val_dataloader = DataLoader(val_dataset, batch_size=batch_size)
    val_size = val_dataset.size()
    correct_count = 0
    for val_batch_idx, val_samples in enumerate(val_dataloader):
        x_val, y_val = val_samples
        
        net_out = net(x_val)
        pred = net_out.argmax(dim = 1)
        correct_count += (pred == y_val).float().sum()
        
        val_loss = loss_func(net_out, y_val)

    print('Epoch: {} - Accuracy: {:.6f}'.format(epoch, correct_count/val_size))
    if epoch % 10 == 0:
        torch.save(net.state_dict(), "_".join([weight_path+"weight", str(epoch), 'model.pth']))


  0%|          | 0/101 [00:00<?, ?it/s]/Users/daehyeon/Desktop/수업/컴퓨터비전/final/FCN.py:20: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)
  2%|▏         | 2/101 [00:00<00:08, 11.74it/s]

Epoch: 0 - Accuracy: 0.343874
Epoch: 1 - Accuracy: 0.343874
Epoch: 2 - Accuracy: 0.361660


  6%|▌         | 6/101 [00:00<00:07, 12.03it/s]

Epoch: 3 - Accuracy: 0.426877
Epoch: 4 - Accuracy: 0.559289
Epoch: 5 - Accuracy: 0.719368


  8%|▊         | 8/101 [00:00<00:08, 11.49it/s]

...

Epoch: 96 - Accuracy: 0.998024
Epoch: 97 - Accuracy: 0.998024


 99%|█████████▉| 100/101 [00:11<00:00,  8.42it/s]

Epoch: 98 - Accuracy: 0.998024
Epoch: 99 - Accuracy: 0.998024


100%|██████████| 101/101 [00:11<00:00,  8.76it/s]

Epoch: 100 - Accuracy: 0.998024


In [22]:
pretrained_net = FullyConnectedNet()
pretrained_net.load_state_dict(torch.load(weight_path+"weight_100_model.pth"))

test1_list = [0,1,2,0,1,2,0,1,2,0,1,2,0,1,2,0,1,2,0,1,2,0,1,2,0,1,2,0,1,2,0,1,2,0,1,2,0,1,2,0,1,2]
pred = pretrained_net(torch.FloatTensor(test1_list))

int(pred.argmax())

0